In [1]:
import chgnet
import numpy as np
from pymatgen.core import Structure

In [2]:
from pymatgen.io.gaussian import GaussianOutput


In [6]:
path = "/home/kazuki/project/sample_chgnet/data/dvb_dispersion_bp86_d3zero.out"
gout=GaussianOutput(path)
vars(gout)


{'filename': '/home/kazuki/project/sample_chgnet/data/dvb_dispersion_bp86_d3zero.out',
 'properly_terminated': True,
 'is_pcm': False,
 'stationary_type': 'Minimum',
 'corrections': {},
 'energies': [-382.328035056],
 'pcm': None,
 'errors': [],
 'Mulliken_charges': {1: ['C', -0.0056],
  2: ['C', -0.083571],
  3: ['C', -0.081354],
  4: ['C', -0.0056],
  5: ['C', -0.083571],
  6: ['C', -0.081354],
  7: ['H', 0.083584],
  8: ['H', 0.084257],
  9: ['H', 0.083584],
  10: ['H', 0.084257],
  11: ['C', -0.080568],
  12: ['C', -0.163971],
  13: ['H', 0.083456],
  14: ['H', 0.082983],
  15: ['H', 0.080784],
  16: ['C', -0.080568],
  17: ['C', -0.163971],
  18: ['H', 0.083456],
  19: ['H', 0.082983],
  20: ['H', 0.080784]},
 'link0': {},
 'cart_forces': [],
 'frequencies': [],
 'eigenvalues': {<Spin.up: 1>: [-9.7435,
   -9.74344,
   -9.73224,
   -9.73224,
   -9.73175,
   -9.73173,
   -9.73057,
   -9.73045,
   -9.71621,
   -9.71621,
   -0.7437,
   -0.6838,
   -0.65131,
   -0.63622,
   -0.60399,
 

In [37]:
gout_dict=gout.as_dict()
energy=gout_dict["output"]["final_energy"]
print(energy)
structure=gout_dict["output"]["molecule"]
print(structure)
type(structure)
print(gout.final_structure)
print(type(gout.final_structure))

-382.328035056
{'@module': 'pymatgen.core.structure', '@class': 'Molecule', 'charge': 0.0, 'spin_multiplicity': 1, 'sites': [{'name': 'C', 'species': [{'element': 'C', 'occu': 1}], 'xyz': [1.208103, 0.697409, 0.0], 'properties': {}, 'label': 'C'}, {'name': 'C', 'species': [{'element': 'C', 'occu': 1}], 'xyz': [-0.000235, 1.394826, 0.0], 'properties': {}, 'label': 'C'}, {'name': 'C', 'species': [{'element': 'C', 'occu': 1}], 'xyz': [-1.208103, 0.69749, 0.0], 'properties': {}, 'label': 'C'}, {'name': 'C', 'species': [{'element': 'C', 'occu': 1}], 'xyz': [-1.208103, -0.697409, 0.0], 'properties': {}, 'label': 'C'}, {'name': 'C', 'species': [{'element': 'C', 'occu': 1}], 'xyz': [0.000235, -1.394826, 0.0], 'properties': {}, 'label': 'C'}, {'name': 'C', 'species': [{'element': 'C', 'occu': 1}], 'xyz': [1.208103, -0.69749, 0.0], 'properties': {}, 'label': 'C'}, {'name': 'H', 'species': [{'element': 'H', 'occu': 1}], 'xyz': [-1.5e-05, 2.494481, 0.0], 'properties': {}, 'label': 'H'}, {'name': '

In [52]:
from pymatgen.core.structure import Molecule, Structure
from pymatgen.core.lattice import Lattice

# 例：molecule は既に定義された Molecule オブジェクトです。
molecule = gout.final_structure  # Moleculeオブジェクトを定義するか、既存のものを使用

# 十分に大きな単位格子を作成（例：ある程度大きい立方体の格子）
lattice = Lattice.cubic(20)

# Structure オブジェクトの生成
gout_structure = Structure(lattice, molecule.species, molecule.cart_coords)

# 結果を確認
# print(gout_structure)

pred = chgnet.predict_structure(gout_structure)
print(pred["e"]*20)
print(gout.final_energy)
print(type(molecule.species))
print(len(molecule.species))
print(type(molecule.cart_coords))

-44.721126556396484
-382.328035056
<class 'list'>
20
<class 'numpy.ndarray'>


In [40]:
# Structure to CHGNet graph
from chgnet.graph import CrystalGraphConverter

converter = CrystalGraphConverter()
graph=converter(structure)

In [41]:
try:
    from chgnet import ROOT

    lmo = Structure.from_file(f"{ROOT}/examples/mp-18767-LiMnO2.cif")
except Exception:
    from urllib.request import urlopen

    url = "https://raw.githubusercontent.com/CederGroupHub/chgnet/main/examples/mp-18767-LiMnO2.cif"
    cif = urlopen(url).read().decode("utf-8")
    lmo = Structure.from_str(cif, fmt="cif")

structures, energies_per_atom, forces, stresses, magmoms = [], [], [], [], []
from chgnet.model import CHGNet
chgnet = CHGNet.load()

for _ in range(100):
    structure = lmo.copy()
    # stretch the cell by a small amount
    structure.apply_strain(np.random.uniform(-0.1, 0.1, size=3))
    # perturb all atom positions by a small amount
    structure.perturb(0.1)
    # print(structure)

    pred = chgnet.predict_structure(structure)

    structures.append(structure)
    energies_per_atom.append(pred["e"] + np.random.uniform(-0.1, 0.1, size=1))
    forces.append(pred["f"] + np.random.uniform(-0.01, 0.01, size=pred["f"].shape))
    stresses.append(
        pred["s"] * -10 + np.random.uniform(-0.05, 0.05, size=pred["s"].shape)
    )
    magmoms.append(pred["m"] + np.random.uniform(-0.03, 0.03, size=pred["m"].shape))

CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cuda:0
Full Formula (Li2 Mn2 O4)
Reduced Formula: LiMnO2
abc   :   2.596978   4.436240   5.657297
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (8)
  #  SP           a         b         c
---  ----  --------  --------  --------
  0  Li+   0.521846  0.508319  0.366738
  1  Li+   0.970395  0.011771  0.613726
  2  Mn3+  0.494556  0.478548  0.868073
  3  Mn3+  0.010058  0.020902  0.132006
  4  O2-   0.473926  0.001354  0.373788
  5  O2-   0.972155  0.493708  0.087345
  6  O2-   0.476212  0.994103  0.914595
  7  O2-   0.00251   0.512222  0.653983
Full Formula (Li2 Mn2 O4)
Reduced Formula: LiMnO2
abc   :   3.013200   4.227715   6.175716
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (8)
  #  SP           a         b         c
---  ----  --------  --------  --------
  0  Li+   0.476198  0.505138  0.369029
  1  Li+   0.96758   0.999053  0.61684

In [50]:
structures[0]
print(type(forces))

<class 'list'>


In [18]:
from chgnet.data.dataset import StructureData, get_train_val_test_loader

In [26]:
dataset = StructureData(
    structures=structures,
    energies=energies_per_atom,
    forces=forces,
    stresses=None,  # can be None
    magmoms=None,  # can be None
)
train_loader, val_loader, test_loader = get_train_val_test_loader(
    dataset, batch_size=8, train_ratio=0.9, val_ratio=0.05
)

100 structures imported


In [27]:
from chgnet.model import CHGNet
from chgnet.trainer import Trainer

# Load pretrained CHGNet
chgnet = CHGNet.load()

CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cuda:0


In [28]:
# Optionally fix the weights of some layers
for layer in [
    chgnet.atom_embedding,
    chgnet.bond_embedding,
    chgnet.angle_embedding,
    chgnet.bond_basis_expansion,
    chgnet.angle_basis_expansion,
    chgnet.atom_conv_layers[:-1],
    chgnet.bond_conv_layers,
    chgnet.angle_layers,
]:
    for param in layer.parameters():
        param.requires_grad = False

In [31]:
# Define Trainer
trainer = Trainer(
    model=chgnet,
    targets="ef",
    optimizer="Adam",
    scheduler="CosLR",
    criterion="MSE",
    epochs=5,
    learning_rate=1e-2,
    use_device="cpu",
    print_freq=6,
)

In [32]:
trainer.train(train_loader, val_loader, test_loader)

Begin Training: using cpu device
training targets: ef
Epoch: [0][1/12] | Time (1.332)(0.080) | Loss 0.0020(0.0020) | MAE e 0.036(0.036)  f 0.005(0.005)  
Epoch: [0][6/12] | Time (0.638)(0.040) | Loss 0.0307(0.0206) | MAE e 0.155(0.109)  f 0.045(0.035)  
Epoch: [0][12/12] | Time (0.530)(0.033) | Loss 0.0051(0.0166) | MAE e 0.054(0.092)  f 0.030(0.037)  
*   e_MAE (0.059) 	f_MAE (0.033) 	
Epoch: [1][1/12] | Time (0.449)(0.000) | Loss 0.0077(0.0077) | MAE e 0.051(0.051)  f 0.037(0.037)  
Epoch: [1][6/12] | Time (0.471)(0.000) | Loss 0.0080(0.0069) | MAE e 0.049(0.048)  f 0.037(0.036)  
Epoch: [1][12/12] | Time (0.434)(0.000) | Loss 0.0109(0.0092) | MAE e 0.023(0.055)  f 0.058(0.037)  
*   e_MAE (0.045) 	f_MAE (0.049) 	
Epoch: [2][1/12] | Time (0.559)(0.000) | Loss 0.0128(0.0128) | MAE e 0.039(0.039)  f 0.069(0.069)  
Epoch: [2][6/12] | Time (0.467)(0.000) | Loss 0.0078(0.0100) | MAE e 0.056(0.054)  f 0.041(0.046)  
Epoch: [2][12/12] | Time (0.418)(0.000) | Loss 0.0036(0.0079) | MAE e 0.03